# day 20

https://adventofcode.com/2019/day/20

In [1]:
import logging
import logging.config
import os

import yaml

In [4]:
with open('../logging.yaml') as fp:
    logging_config = yaml.load(fp, Loader=yaml.FullLoader)

logging.config.dictConfig(logging_config)

In [5]:
FNAME = os.path.join('data', 'day20.txt')

LOGGER = logging.getLogger('day20')

## part 1

### problem statement:

#### loading data

In [7]:
test_0 = """         A           
         A           
  #######.#########  
  #######.........#  
  #######.#######.#  
  #######.#######.#  
  #######.#######.#  
  #####  B    ###.#  
BC...##  C    ###.#  
  ##.##       ###.#  
  ##...DE  F  ###.#  
  #####    G  ###.#  
  #########.#####.#  
DE..#######...###.#  
  #.#########.###.#  
FG..#########.....#  
  ###########.#####  
             Z       
             Z       """, 23

test_1 = """                   A               
                   A               
  #################.#############  
  #.#...#...................#.#.#  
  #.#.#.###.###.###.#########.#.#  
  #.#.#.......#...#.....#.#.#...#  
  #.#########.###.#####.#.#.###.#  
  #.............#.#.....#.......#  
  ###.###########.###.#####.#.#.#  
  #.....#        A   C    #.#.#.#  
  #######        S   P    #####.#  
  #.#...#                 #......VT
  #.#.#.#                 #.#####  
  #...#.#               YN....#.#  
  #.###.#                 #####.#  
DI....#.#                 #.....#  
  #####.#                 #.###.#  
ZZ......#               QG....#..AS
  ###.###                 #######  
JO..#.#.#                 #.....#  
  #.#.#.#                 ###.#.#  
  #...#..DI             BU....#..LF
  #####.#                 #.#####  
YN......#               VT..#....QG
  #.###.#                 #.###.#  
  #.#...#                 #.....#  
  ###.###    J L     J    #.#.###  
  #.....#    O F     P    #.#...#  
  #.###.#####.#.#####.#####.###.#  
  #...#.#.#...#.....#.....#.#...#  
  #.#####.###.###.#.#.#########.#  
  #...#.#.....#...#.#.#.#.....#.#  
  #.###.#####.###.###.#.#.#######  
  #.#.........#...#.............#  
  #########.###.###.#############  
           B   J   C               
           U   P   P               """, 58

In [10]:
def load_data(fname=FNAME):
    with open(fname) as fp:
        return fp.read()

In [17]:
HALL = '.'
WALL = '#'
EMPTY = ' '

import string

def map_to_grid(s):
    return {(i, j): c
            for (i, row) in enumerate(s.split('\n'))
            if row.strip()
            for (j, c) in enumerate(row)
            if c in (HALL + string.ascii_uppercase)}

In [50]:
import networkx as nx

def grid_to_nx(grid):
    g = nx.Graph()
    # handle the halls first
    for ij, c in grid.items():
        if c == HALL:
            g.add_node(ij)
    
    seen_teleportals = set()
    for (i, j), c in grid.items():
        # LR
        l = (i, j - 1)
        cl = grid.get(l, '~')
        r = (i, j + 1)
        cr = grid.get(r, '~')
        
        # UD
        u = (i - 1, j)
        cu = grid.get(u, '~')
        d = (i + 1, j)
        cd = grid.get(d, '~')
        
        if c == HALL:
            for nbr in (l, r, u, d):
                if grid.get(nbr, '~') == HALL:
                    g.add_edge((i, j), nbr)
                    
        elif c in string.ascii_uppercase:
            # we are looking for a letter on one side and a . on the other
            # otherwise ignore
            
            if cl in string.ascii_uppercase and cr == '.':
                teleportal = f'{cl}{c}'
                if teleportal in seen_teleportals:
                    teleportal2 = f'{teleportal}_2'
                    g.add_edge(teleportal2, r)
                    g.add_edge(teleportal, teleportal2)
                else:
                    g.add_edge(teleportal, r)
                    seen_teleportals.add(teleportal)
            elif cl == '.' and cr in string.ascii_uppercase:
                teleportal = f'{c}{cr}'
                if teleportal in seen_teleportals:
                    teleportal2 = f'{teleportal}_2'
                    g.add_edge(teleportal2, r)
                    g.add_edge(teleportal, teleportal2)
                else:
                    g.add_edge(teleportal, r)
                    seen_teleportals.add(teleportal)
                g.add_edge((c, cr), l)
            
            if cu in string.ascii_uppercase and cd == '.':
                teleportal = f'{cu}{c}'
                if teleportal in seen_teleportals:
                    teleportal2 = f'{teleportal}_2'
                    g.add_edge(teleportal2, r)
                    g.add_edge(teleportal, teleportal2)
                else:
                    g.add_edge(teleportal, r)
                    seen_teleportals.add(teleportal)
            elif cu == '.' and cd in string.ascii_uppercase:
                teleportal = f'{c}{cd}'
                if teleportal in seen_teleportals:
                    teleportal2 = f'{teleportal}_2'
                    g.add_edge(teleportal2, r)
                    g.add_edge(teleportal, teleportal2)
                else:
                    g.add_edge(teleportal, r)
                    seen_teleportals.add(teleportal)

    return g

In [51]:
def map_to_nx(s):
    return grid_to_nx(map_to_grid(s))

In [52]:
g = map_to_nx(test_0[0])

In [53]:
g.edges()

EdgeView([((2, 9), (3, 9)), ((3, 9), (3, 10)), ((3, 9), (4, 9)), ((3, 10), (3, 11)), ((3, 11), (3, 12)), ((3, 12), (3, 13)), ((3, 13), (3, 14)), ((3, 14), (3, 15)), ((3, 15), (3, 16)), ((3, 16), (3, 17)), ((3, 17), (4, 17)), ((4, 9), (5, 9)), ((4, 17), (5, 17)), ((5, 9), (6, 9)), ((5, 17), (6, 17)), ((6, 17), (7, 17)), ((7, 17), (8, 17)), ((8, 2), 'BC_2'), ((8, 2), (8, 3)), ((8, 3), (8, 4)), ((8, 4), (9, 4)), ((8, 17), (9, 17)), ((9, 4), (10, 4)), ((9, 17), (10, 17)), ((10, 4), (10, 5)), ((10, 5), (10, 6)), ((10, 6), ('D', 'E')), ((10, 17), (11, 17)), ((11, 17), (12, 17)), ((12, 11), (13, 11)), ((12, 17), (13, 17)), ((13, 2), 'DE_2'), ((13, 2), (13, 3)), ((13, 3), (14, 3)), ((13, 11), (13, 12)), ((13, 12), (13, 13)), ((13, 13), (14, 13)), ((13, 17), (14, 17)), ((14, 3), (15, 3)), ((14, 13), (15, 13)), ((14, 17), (15, 17)), ((15, 2), 'FG_2'), ((15, 2), (15, 3)), ((15, 13), (15, 14)), ((15, 13), (16, 13)), ((15, 14), (15, 15)), ((15, 15), (15, 16)), ((15, 16), (15, 17)), ('AA', (1, 10)),

#### function def

In [66]:
def q_1(data):
    LOGGER.debug(f'\n{data}')
    g = map_to_nx(data)
    return nx.shortest_path_length(g, 'AA', 'ZZ')

#### tests

In [67]:
def test_q_1():
    LOGGER.setLevel(logging.DEBUG)
    t, a = test_0
    assert q_1(t) == a
    LOGGER.setLevel(logging.INFO)

In [68]:
test_q_1()

2019-12-20 00:31:22,884 DEBUG    [day20.q_1:2] 
         A           
         A           
  #######.#########  
  #######.........#  
  #######.#######.#  
  #######.#######.#  
  #######.#######.#  
  #####  B    ###.#  
BC...##  C    ###.#  
  ##.##       ###.#  
  ##...DE  F  ###.#  
  #####    G  ###.#  
  #########.#####.#  
DE..#######...###.#  
  #.#########.###.#  
FG..#########.....#  
  ###########.#####  
             Z       
             Z       


NetworkXNoPath: No path between AA and ZZ.

#### answer

In [ ]:
q_1(load_data())

## part 2

### problem statement:

#### function def

In [ ]:
def q_2(data):
    return False

#### tests

In [ ]:
def test_q_2():
    LOGGER.setLevel(logging.DEBUG)
    assert q_2(test_data) == True
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_2()

#### answer

In [ ]:
q_2(load_data())

fin

In [70]:
from collections import defaultdict
from string import ascii_uppercase

import networkx as nx


def get_nbs(point):
    """Get all four neighbouring points."""
    return [(point[0] + dx, point[1] + dy) for dx, dy in zip([0, 1, 0, -1], [-1, 0, 1, 0])]


in1 = load_data()[:-1].split('\n')

grid = defaultdict(lambda: '#')
W, H = len(in1[0]), len(in1)

for y in range(H):
    for x in range(W):
        grid[(x, y)] = in1[y][x] if in1[y][x] != ' ' else '#'

portals = defaultdict(list)  # matches the portal names to their coordinates like portals['XY'] = [(x1,y1), (x2, y2)]
G = nx.Graph()
start = end = None
for y in range(1, H - 1):
    for x in range(1, W - 1):
        symbol = grid[(x, y)]
        if symbol in ascii_uppercase:
            nbs = [(a, b) for a, b in get_nbs((x, y)) if grid[(a, b)] != '#']
            if len(nbs) == 2:
                # letter and pathway found
                if grid[nbs[0]] in ascii_uppercase:
                    letter, pad = nbs
                else:
                    pad, letter = nbs
                key = ''.join(sorted(symbol + grid[letter]))  # sort portal name
                portals[key].append(pad)
                if key == 'AA':
                    start = pad
                elif key == 'ZZ':
                    end = pad
        elif symbol == '.':
            G.add_node((x, y))
            nbs = [(a, b) for a, b in get_nbs((x, y)) if grid[(a, b)] == '.']
            for nb in nbs:
                G.add_edge((x, y), nb)  # connect pathways

for pads in portals.values():
    if len(pads) == 2:
        G.add_edge(pads[0], pads[1])  # connect portals

ansa = nx.shortest_path_length(G, start, end)
print("Number of steps to get from AA to ZZ:", ansa)
# puzzle.answer_a = ansa

Number of steps to get from AA to ZZ: 410


In [71]:
from collections import defaultdict
from string import ascii_uppercase

import networkx as nx


def get_nbs(point):
    """Get all four neighbouring points."""
    return [(point[0] + dx, point[1] + dy) for dx, dy in zip([0, 1, 0, -1], [-1, 0, 1, 0])]


load_data()[:-1].split('\n')

grid = defaultdict(lambda: '#')
W, H = len(in1[0]), len(in1)

for y in range(H):
    for x in range(W):
        grid[(x, y)] = in1[y][x] if in1[y][x] != ' ' else '#'

portals = defaultdict(list)  # matches the portal names to their coordinates like portals['XY'] = [(x1,y1), (x2, y2)]
G = nx.Graph()
levels = 30  # maximum allowed recursion depth
start = end = None
for y in range(1, H - 1):
    for x in range(1, W - 1):
        symbol = grid[(x, y)]
        if symbol in ascii_uppercase:
            nbs = [(a, b) for a, b in get_nbs((x, y)) if grid[(a, b)] != '#']
            if len(nbs) == 2:
                # letter and pathway found
                if grid[nbs[0]] in ascii_uppercase:
                    letter, pad = nbs
                else:
                    pad, letter = nbs
                key = ''.join(sorted(symbol + grid[letter]))  # sort portal name
                portals[key].append(pad)
                if key == 'AA':
                    start = pad
                elif key == 'ZZ':
                    end = pad
        elif symbol == '.':
            for i in range(levels):
                G.add_node((x, y, i))  # create the node on each level
            nbs = [(a, b) for a, b in get_nbs((x, y)) if grid[(a, b)] == '.']
            for nb in nbs:
                for i in range(levels):
                    G.add_edge((x, y, i), (*nb, i))  # connect the pathways on each level

for pads in portals.values():
    if len(pads) == 2:
        if pads[0][0] in [2, W - 3] or pads[0][1] in [2, H - 3]:
            outer, inner = pads
        else:
            inner, outer = pads
        for i in range(levels - 1):
            # inner portals lead to the outer portals on the next level and outer to inner on the previous level
            G.add_edge((*inner, i), (*outer, i + 1))
            G.add_edge((*outer, i + 1), (*inner, i))


ansb = nx.shortest_path_length(G, (*start, 0), (*end, 0))  # specify that we want to start and end on level 0
print("Number of steps to get from AA to ZZ on level 0:", ansb)
# puzzle.answer_b = ansb

Number of steps to get from AA to ZZ on level 0: 5084
